# Beam Loading 

# 1. In the linear regime (beam driver)

Notebook developed by Lance Hildebrand using scripts made by Yujian Zhao

## Theory

In general, each slice of a beam in a wakefield will be accelerated at different rates due to the variation in the accelerating field, $E_z$. This will lead energy spread growth in the beam, which usually isn't desirable. The way to avoid this issue is called beam loading the wake.

In this notebook, we will focus on the linear regime, specifically a two-bunch scenario. Here the idea is quite simple. The total $E_z$ felt by the witness bunch will be the field created by the driver plus the one created by itself, via superposition. Therefore, for given drive bunch parameters, we can calulate the wake and therefore we can tailor a witness beam with a profile that will create a wake that will flatten the total field (first shown in Katsouleas et al., 1986). We know in the linear regime the wake behind the driver goes as $\cos\xi$. So we need a witness bunch with a wake that has an cosine part $\pi$ out of phase with the driver to cancel the $\xi$ dependence of the wake and a constant part equal to the value of the driver wake at the head of the witness bunch. It turns out a triangular shaped bunch can create such a wake. Gaussian beams can load a wake approximately but not perfectly. Now let's look at the math. For all of the following we will use normalized units.

The triangular bunch has four parameters we can tune: the max charge density, the transverse spot size, and the location of the head and the tail of the bunch. The profile is of the form (using $\xi=ct-z$)

$\displaystyle \rho=\rho_0\frac{\xi_f-\xi}{\xi_f-\xi_0}$.

We can calculate the on-axis field from this exactly (c.f. linear pwfa notebook),

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0} \int_{\xi_0}^\xi d\xi' (\xi_f-\xi') \cos (\xi-\xi')$,

where $R_w(0)$ is the radial function on-axis for the witness beam defined in the linear pwfa notebook. We have

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0}\left[ (\xi_f-\xi_0)\sin (\xi-\xi_0)+\cos (\xi-\xi_0)-1 \right]$.

We can use trig identites to separate $\sin\xi$ and $\cos\xi$ terms,

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0}\left[ \left(\sin\xi_0+(\xi_f-\xi_0)\cos\xi_0\right)\sin\xi+\left(\cos\xi_0-(\xi_f-\xi_0)\sin\xi_0\right)\cos\xi -1 \right]$.

Alright, now we want to choose parameters such that the $\sin\xi$ term goes away and the coefficient on the $\cos\xi$ is opposite of the driver amplitude, call it $E_0$ for now. The astute observer will notice we have one more constraint that the constant here needs to be $E_0\cos\xi_0$. We will see this is redundant with the second one. The constraints are

$\displaystyle (\xi_f-\xi_0)\cos\xi=-\sin\xi$

$\displaystyle \cos\xi-(\xi_f-\xi_0)\sin\xi_0=\frac{(\xi_f-\xi_0)E_0}{R_w(0)\rho_0}$,

or, simplified,

$\displaystyle \xi_f=\xi_0-\tan\xi_0$

$\displaystyle \frac{\rho_0 R_w(0)}{E_0}=-\sin\xi_0$.

We have one degree of freedom. So say for example we know the beam separation, then these constraints will tell us how long and how much charge we need to flatten the wake. 

An important thing to note is that if we are loading with an electron beam we need the slope of the wake to be negative at the head of the beam, i.e. $\pi/2<\xi_0<\pi$. If you are loading with positive charge you would need to put it in $3\pi/2<\xi_0<2\pi$.

For reference,

$\displaystyle E_0=\rho_{0}R(0)\sqrt{2\pi}\sigma_{z}e^{-\sigma_{z}^2/2}$

for a Gaussian bunch, and

$\displaystyle R(0)=\frac{1}{2}\sigma_r^2 e^{\sigma_r^2/2}\Gamma\left(0,\sigma_r^2/2\right)$

for a transverse Gaussian profile, where $\Gamma(n,x)=\int_x^\infty t^{n-1}e^{-t}dt$.

The next cell will allow you to explore beam loading with different drive and witness bunch parameters using these analytic formulas. The followings cells will allow you to use your predictions to set up and run a QuickPIC simulation to confirm the parameters you calculated load the wake.

In [7]:
from importlib import reload
import wakeplot
wakeplot = reload(wakeplot)

#Specify xi dimensions (shouldn't have to change this)
indz=8     #number of cells
box_z=[-3.0,10.0]     #min and max xi values

#Specify drive beam parameters (all in normalized units)
nb_driver=0.005   #beam denisty of driver normalized to the plasma density
z_center_driver=0.0     #location of the center of the drive beam profile
sigma_r_driver=0.5     #transv. spot size of the driver 
sigma_z_driver=0.4     #long. spot size of the driver

#Specify witness beam parameters
#### for positrons just specify nb_witness to be negative (note on the second plot the slider will still show as positive even though it's really negative)
nb_witness=0.062     #beam denisty of witness beam normalized to the plasma density (will result in error if density is out of the appropriate range)
z_front_witness=2.0    #location of the front of th\e witness beam (between pi/2 and pi)
z_back_witness=z_front_witness+0.8     #location of the back of the witness beam
sigma_r_witness=0.5   #transv. spot size of the witness beam

use_theory=True      #calculate the two free parameters from theory given the slider parameter. (if true the wake will always be flattened)
use_fixed_length=True     #for first plot (if use theory is false), this will force the beam to maintain the initially specified length

wakeplot.z1_interact(box_z,indz,nb_driver,z_center_driver,sigma_r_driver,sigma_z_driver,
                     nb_witness,sigma_r_witness,z_front_witness,z_back_witness,use_fixed_length,use_theory)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='z1_w', max=3.1, min=1.6, st…

interactive(children=(SelectionSlider(continuous_update=False, description='nb_w', options=(('4.582e-04', 0.00…

## Simulation
### 1.1 Make an input file

In [2]:
from importlib import reload
import qphelper
qphelper = reload(qphelper)
qphelper.makeWidgetsForInput()

interactive(children=(Text(value='qpinput_linear.json', description='Template Input File:', layout=Layout(widt…

### 1.2 Run QuickPIC simulation

In [3]:
import quickpic
qphelper.deleteAllOutput()
rundir = 'linear'
fname = 'qpinput.json'
quickpic.runqpic(rundir=rundir,inputfile=fname)

quickpic completed normally


### 1.3 Visualize the output data

In [4]:
qphelper = reload(qphelper)
rundir = 'linear'
qphelper.makeplot(rundir)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='x_position', max=3.0, min=-…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='x_position', max=3.0, min=-…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='x_position', max=3.0, min=-…

### 4. Calculate the acceleration efficiency

The acceleration efficiency is evaluated through the ratio of the energy gain of witness beam to the energy loss of driver beam.

$\displaystyle \eta=\frac{\sum_i [q_i(\gamma_{i,t_1}-\gamma_{i,t_0})]_\text{witness}}{\sum_i [q_i(\gamma_{i,t_0}-\gamma_{i,t_1})]_\text{driver}}$

In [5]:
qphelper = reload(qphelper)
qphelper.get_efficiency(rundir)

The acceleration efficiency is:  0.3499054764543319


# 2. In the nonlinear blowout regime 

In this notebook, we are going to introduce the beam loading theory (add citation) in the nonlinear regime of PWFA and compare it with the QuickPIC simulations.

From Weilu's theory (add citation), in the untrarelativistic limit, the pseudo-potential inside the ion channel is:

$$\psi(r,\xi) \approx \frac{1}{4}(r_b(\xi)^2 - r^2)$$

where $r_b(\xi)$ is the boundary of the ion channel, which is described by the differential equation:

$$r_br_b''(\xi) + 2[r_b'(\xi)]^2 + 1 = \frac{4\lambda(\xi)}{r_b(\xi)^2} \tag{1} $$

where $\lambda(\xi) = \int_0^\infty rn_b dr$, is the charge per unit length of an electron beam (drive or trailing beam) divided by $2\pi$. Here we are only interested in the back half of the bubble (where beam loading occurs), we can interpret $\lambda(\xi)$ as the charge per unit length of the trailing beam divided by $2\pi$. Also, we set up the coordinate system as the following: (add citation)

In beam loading, we are interested in the accelerating field, which is calculated by taking the longitudinal derivative with respect to $\psi$:

$$E_z = \frac{\partial \psi}{\partial \xi} = \frac{1}{2}r_b r_b'(\xi)$$

So the idea is to solve for $r_b$ from equation (1), then plug it in the above equation to obtain $E_z$. So now let us solve equation (1). We change variable from $\xi$ to $r_b$, so $r_b''(\xi) = \frac{dr_b'}{d\xi} = \frac{dr_b'}{dr_b} \frac{dr_b}{d\xi} = r_b' \frac{dr_b'}{dr_b}$, so equation (1) becomes:

$$r_br_b' \frac{dr_b'}{dr_b} + 2r_b'^2 + 1 = \frac{4\lambda}{r_b^2} \tag{2}$$

I changed the variable from $\xi$ to $r_b$ implicitly so now everything in the above equation is a function of $r_b$. The trick to solve this equation is to realize $r_b' \frac{dr_b'}{dr_b} = \frac{1}{2} \frac{d(r_b'^2)}{dr_b}$, so we can make a substitution $u = r_b'^2$. Equation (2) reduces to:

$$\frac{1}{2} r_b \frac{du}{dr_b} + 2u + 1 = \frac{4\lambda}{r_b^2}$$

This is a first order differential equation. We can multiply $2 r_b^3$ on both sides:

$$\frac{d}{dr_b}(u r_b^4)= 8r_b\lambda - 2r_b^3$$

So:

$$r_b'^2 = u = \frac{8\int\lambda(r_b)r_bdr_b - \frac{1}{2}r_b^4}{r_b^4}$$

At the back of the bubble ($\xi \geq 0$), $r_b' = dr_b / d\xi \leq 0$. So:

$$r_b' = -\sqrt{\frac{8\int\lambda(r_b)r_bdr_b - \frac{1}{2}r_b^4}{r_b^4}}$$

So finally, we get:

$$E_z = \frac{1}{2}r_b r_b' = -\frac{r_b}{2\sqrt{2}}\sqrt{\frac{16\int\lambda(r_b)r_bdr_b}{r_b^4}-1} \tag{3}$$

When $0 \leq \xi \leq \xi_s$, $\lambda = 0$. So the indefinite integral is equal to a constant: $E_z = -\frac{r_b}{2\sqrt{2}}\sqrt{\frac{C}{r_b^4}-1}$. We determine the constant $C$ using the condition that $E_z = 0$ at $r_b = R_b$ (because $r_b'(\xi = 0) = 0$ here). We can get $C = R_b^4$. So:

$$E_z = -\frac{r_b}{2\sqrt{2}}\sqrt{\frac{R_b^4}{r_b^4}-1} \;\;\; (0 \leq \xi \leq \xi_s) \tag{4}$$

When $\xi > \xi_s$, $\lambda$ is determined by the trailing beam. We want $E_z$ to be a constant (all the way until the end of the ion channel) so that the energy spread of the trailing beam is minimized. We denote this constant by $-E_s$. So we have:

$$-E_s = -\frac{r_b}{2\sqrt{2}}\sqrt{\frac{16\int\lambda(r_b)r_bdr_b}{r_b^4}-1}$$

We can get:

$$\lambda(r_b) = E_s^2 + \frac{1}{4}r_b^2 \tag{5}$$

So we need to figure out: 1. The constant accelerating field: $-E_s$. 2. The bubble boundary: $r_b$ when $\xi \geq \xi_s$

* $E_s$

We require $E_z$ to be continuous at $\xi = \xi_s$ (or $r_b = r_s$). So from equation (4) we have:

$$E_s = \frac{r_s}{2\sqrt{2}}\sqrt{\frac{R_b^4}{r_s^4}-1} \tag{6}$$

* $r_b \; (\xi \geq \xi_s)$

We know the relation between the accelerating field and the bubble boundary:

$$-E_s = \frac{1}{2}r_b(\xi)r_b'(\xi) = \frac{1}{4}\frac{d(r_b^2)}{d\xi}$$

Using the boundary condition: At $\xi = \xi_s$, $r_b = r_s$, we get:

$$r_b^2 = r_s^2 - 4E_s(\xi - \xi_s) \tag{7}$$

Setting $r_b = 0$ we can get the end of the ion channel: $\xi = \xi_s + \frac{r_s^2}{4E_s}$.

Now we can plug (6) and (7) into (5) to get the expression for the linear charge density of the witness beam (divided by $2\pi$). To make the expression simpler, we express $r_s^2$ in terms of $E_s$ first, using equation (6):

$$r_s^2 = \sqrt{R_b^4 + 16E_s^4} - 4E_s^2 \tag{8}$$

Then equation (5) becomes:

$$\lambda(\xi) = \sqrt{E_s^4 + \frac{R_b^4}{16}} - E_s(\xi - \xi_s)$$

The total charge on the trailing beam is:

$$Q_s = 2\pi \int_{\xi_s}^{\xi_s + \frac{r_s^2}{4E_s}} \lambda(\xi) d\xi = 2\pi \Big[\sqrt{E_s^4 + \frac{R_b^4}{16}} - \frac{E_s}{2}\frac{r_s^2}{4E_s}\Big]\frac{r_s^2}{4E_s}$$

Use equation (8) and simply, we obtain:

$$Q_s E_s = \frac{\pi R_b^4}{16}$$



### 2.1 Make an input file
* In the same directory as this Jupyter notebook, there must be a template input file (the default name is qpinput_nonlinear.json) and two python files: mkQPinput_BL.py and pyVisQP.py

* We assume azymuthal symmetry, no energy spread and no transverse momentum for the beam.

### First we run a simulation with drive beam only. So make sure we select 'number of beams' (5th line) to be 1.

In [8]:
from importlib import reload
import mkQPinput_BL
mkQPinput_BL = reload(mkQPinput_BL)
mkQPinput_BL.makeWidgetsForInput()

interactive(children=(Text(value='qpinput_nonlinear.json', description='Template Input File:', layout=Layout(w…

### 2.2 Run the simulation

In [11]:
import quickpic
dirname = 'BL_one_bunch'
quickpic.runqpic(rundir=dirname,inputfile='qpinput.json')

quickpic completed normally


### 2.3 Visualize the output data

In [12]:
from importlib import reload
import pyVisQP
pyVisQP = reload(pyVisQP)

dirname = 'BL_one_bunch'

# LineoutDir can be chosen from: None, 'transverse', 'longitudinal'

# Plot the charge density of plasma and beam
filename1 = dirname +'/Species0001/Charge_slice_0001/charge_slice_xz_00000001.h5'
filename2 = dirname +'/Beam0001/Charge_slice_0001/charge_slice_xz_00000001.h5'
filenameList = [filename1,filename2]
scaleList = [1,1]
pyVisQP.makeplot(filenameList,scaleList,LineoutDir = None)


# Plot the accelerating field with transverse and longitudinal lineout
filename = dirname +'/Fields/Ez_slice0001/ezslicexz_00000001.h5'
filenameList = [filename]
pyVisQP.makeplot(filenameList,LineoutDir = 'longitudinal',specify_title = '$E_z \; [mc\omega_p / e]$')

interactive(children=(FloatRangeSlider(value=(-5.0, 0.0), continuous_update=False, description='Colorbar:', ma…

interactive(children=(FloatRangeSlider(value=(-1.0, 1.0), continuous_update=False, description='Colorbar:', ma…

### 2.4 Rerun QuickPIC simulation for Both drive beam and witness beam 

### Now we should go back to step 1, run the cell, select 'number of beams' (5th line) to be 2, then click 'Make New Input!' Then we run the cell below.

In [15]:
import quickpic
dirname = 'BL_two_bunch'
quickpic.runqpic(rundir=dirname,inputfile='qpinput.json')

quickpic completed normally


### 5. Visualize the results ( Both drive beam and witness beam )

In [16]:
from importlib import reload
import pyVisQP
import qphelper
pyVisQP = reload(pyVisQP)

dirname = 'BL_two_bunch'

# LineoutDir can be chosen from: None, 'transverse', 'longitudinal'

# Plot the charge density of plasma and beam
filename1 = dirname +'/Species0001/Charge_slice_0001/charge_slice_xz_00000001.h5'
filename2 = dirname +'/Beam0001/Charge_slice_0001/charge_slice_xz_00000001.h5'
filename3 = dirname +'/Beam0002/Charge_slice_0001/charge_slice_xz_00000001.h5'
filenameList = [filename1,filename2,filename3]
scaleList = [1,1,1]
pyVisQP.makeplot(filenameList,scaleList,LineoutDir = None)


# Plot the accelerating field with transverse and longitudinal lineout
filename = dirname +'/Fields/Ez_slice0001/ezslicexz_00000001.h5'
filenameList = [filename]
pyVisQP.makeplot(filenameList,LineoutDir = 'longitudinal',specify_title = '$E_z \; [mc\omega_p / e]$')

interactive(children=(FloatRangeSlider(value=(-5.0, 0.0), continuous_update=False, description='Colorbar:', ma…

interactive(children=(FloatRangeSlider(value=(-1.0, 1.0), continuous_update=False, description='Colorbar:', ma…

### 2.6 Calculate the Efficiency 

The acceleration efficiency is evaluated through the ratio of the energy gain of witness beam to the energy loss of driver beam.

$\displaystyle \eta=\frac{\sum_i [q_i(\gamma_{i,t_1}-\gamma_{i,t_0})]_\text{witness}}{\sum_i [q_i(\gamma_{i,t_0}-\gamma_{i,t_1})]_\text{driver}}$

In [17]:
qphelper = reload(qphelper)
qphelper.get_efficiency(dirname)

The acceleration efficiency is:  0.407795264326431
